## Customer Churn Prediction

#### mounting google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Loading Dataset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Churn_Modelling.csv')

#### First view

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


#### Number of rows and columns

In [5]:
df.shape

(10000, 14)

#### Handling Null Values

In [6]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

#### Duplicate values

In [7]:
df.duplicated().sum()

0

#### Feature selection

In [8]:
df=df.drop(['RowNumber','Surname','Geography'],axis=1)

In [9]:
df.head()

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,619,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Female,43,2,125510.82,1,1,1,79084.10,0


#### Categorical encoding

In [10]:
df['Gender']=df['Gender'].map({'Female':0,'Male':1})

In [11]:
df.head()

,CustomerId,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15634602,619,0,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,0,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,0,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,0,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,0,43,2,125510.82,1,1,1,79084.10,0


### Correlation Heatmap

In [41]:
import plotly.express as px
corr_matrix = df.corr()

# Create the correlation heatmap
fig = px.imshow(corr_matrix,
                text_auto=True,
                aspect="auto",
                labels=dict(color="Correlation"),
                x=corr_matrix.columns,
                y=corr_matrix.columns,
                color_continuous_scale='inferno',  # Set color scale to 'inferno'
                zmin=-1, zmax=1)

fig.update_layout(title='Correlation Heatmap',
                  xaxis_nticks=36)

# Show the heatmap
fig.show()

In [12]:
df['Gender'].unique()

array([0, 1])

In [13]:
df=df.drop('CustomerId',axis=1)

#### Independent and Dependent Variable

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X=df.drop('Exited',axis=1)
y=df['Exited']

In [16]:
X.shape

(10000, 9)

In [17]:
y.shape

(10000,)

#### Splitting Data into Training and Testing

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=240)

In [19]:
print('x train shape',X_train.shape)
print('x test shape',X_test.shape)
print('y train shape',y_train.shape)
print('y test shape',y_test.shape)

x train shape (7500, 9)
x test shape (2500, 9)
y train shape (7500,)
y test shape (2500,)


In [20]:
from sklearn.preprocessing import StandardScaler

#### Standardizing the data

In [21]:
scaler=StandardScaler()

In [22]:
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [23]:
X_train_scaled

array([[-2.28987422, -1.0957665 , -1.1399744 , ...,  0.64759498,
         0.97056631, -1.32664916],
       [ 0.35380242, -1.0957665 ,  0.3920605 , ..., -1.54417503,
        -1.0303263 , -0.54860212],
       [ 1.23158568,  0.91260319, -0.85271786, ..., -1.54417503,
         0.97056631, -0.85963607],
       ...,
       [ 0.75655003,  0.91260319, -0.46970913, ..., -1.54417503,
        -1.0303263 ,  0.99135019],
       [-0.62724946, -1.0957665 ,  1.15807795, ...,  0.64759498,
         0.97056631,  1.57991669],
       [ 0.73589631,  0.91260319,  0.67931704, ...,  0.64759498,
         0.97056631, -0.10950967]])

#### Importing Deep learning Libraries

In [24]:
import tensorflow

In [25]:
from tensorflow import keras

In [26]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [27]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

#### Regularization techniques to reduce overfitting

In [28]:
from tensorflow.keras.regularizers import l1

In [29]:
input_shape = (X_train_scaled.shape[1],)

#### Setting up the model

#### First hidden layer and activation function

In [30]:
model = Sequential()

# Add input layer and first hidden layer
model.add(Dense(units=128, activation='relu', input_shape=input_shape,kernel_regularizer=l1(0.001)))


#### Second Hidden layer

In [31]:
model.add(Dense(units=64, activation='relu'))

#### Output Layer

In [32]:
model.add(Dense(units=1, activation='sigmoid'))


#### Compiling the model

In [33]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1280      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 9601 (37.50 KB)
Trainable params: 9601 (37.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Fitting training data into model

In [34]:
history = model.fit(X_train_scaled, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2)



Epoch 1/100
188/188 [==============================] - 3s 4ms/step - loss: 0.5445 - accuracy: 0.8173 - val_loss: 0.4880 - val_accuracy: 0.8373
Epoch 2/100
188/188 [==============================] - 1s 4ms/step - loss: 0.4576 - accuracy: 0.8508 - val_loss: 0.4320 - val_accuracy: 0.8453
Epoch 3/100
188/188 [==============================] - 1s 3ms/step - loss: 0.4205 - accuracy: 0.8552 - val_loss: 0.4149 - val_accuracy: 0.8473
Epoch 4/100
188/188 [==============================] - 2s 11ms/step - loss: 0.4040 - accuracy: 0.8552 - val_loss: 0.3924 - val_accuracy: 0.8520
Epoch 5/100
188/188 [==============================] - 3s 16ms/step - loss: 0.3914 - accuracy: 0.8550 - val_loss: 0.3925 - val_accuracy: 0.8507
Epoch 6/100
188/188 [==============================] - 1s 6ms/step - loss: 0.3830 - accuracy: 0.8595 - val_loss: 0.3820 - val_accuracy: 0.8540
Epoch 7/100
188/188 [==============================] - 1s 4ms/step - loss: 0.3786 - accuracy: 0.8600 - val_loss: 0.3850 - val_accuracy: 0.85

In [36]:
y_log=model.predict(X_test_scaled)

79/79 [==============================] - 0s 2ms/step


In [37]:
y_pred=np.where(y_log>0.5,'will churn','will not churn')

In [38]:
y_pred

array([['will not churn'],
       ['will not churn'],
       ['will not churn'],
       ...,
       ['will not churn'],
       ['will not churn'],
       ['will not churn']], dtype='<U14')

In [42]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test accuracy: {test_accuracy}')

79/79 [==============================] - 0s 2ms/step - loss: 0.3791 - accuracy: 0.8492
Test accuracy: 0.8492000102996826


## Here we are getting accuracy of 84.92%